###### Content under Creative Commons Attribution license CC-BY 4.0, code under BSD 3-Clause License © 2022  by D. Koehn, T. Meier and J. Stampa, notebook style sheet by L.A. Barba, N.C. Clementi

In [1]:
# Execute this cell to load the notebook's style sheet, then ignore it
from IPython.core.display import HTML
css_file = '../style/custom.css'
HTML(open(css_file, "r").read())

# Digitale Signalverarbeitung: Faltung und Filterung

## Kapitel 7 Übungen

### Aufgabe 7.1

Zeigen Sie, dass $(\delta \ast f)(\tau) = f(\tau)$ ist. Interpretieren Sie das Ergebnis.

Laut Vorlesung ist die Faltung zweier kontinuierlicher Funktionen definiert als 

\begin{equation}
(\delta \ast f)(\tau)=\int_{-\infty}^{\infty}\delta(t')f(\tau-t')dt'\notag
\end{equation}

Die [Siebeigenschaft der Dirac'schen Deltafunktion](https://de.wikipedia.org/wiki/Delta-Distribution#Eigenschaften) führt dazu, daß das Integral den Funktionswert $f(\tau-t')$ an der Stelle $t'=0$ ergibt. Damit folgt

\begin{equation}
(\delta \ast f)(\tau)=\int_{-\infty}^{\infty}\delta(t')f(\tau-t')dt'=f(\tau)\notag
\end{equation}

was zu zeigen war.

### Aufgabe 7.2

Zeigen Sie, dass eine gleitende Mittelung einer Faltung mit einer Boxcar-Funktion entspricht.

Wurde bereits in [Aufgabe 4.5](https://nbviewer.org/github/daniel-koehn/Geophysikalische-Signalverarbeitung/blob/master/exercises/Ex_04.ipynb) gezeigt.

### Aufgabe 7.3

Definieren Sie eine 200 s lange Wertereihe, die bei 100 s den Wert 1 annimmt und ansonsten Null ist. Der Abtastschritt soll 0.1 s betragen. Filtern Sie die Wertereihe mit einem einseitigen Butterworth-Filter (Ordnung 3), der die folgenden Eigenschaften besitzt:
- Tiefpass, Grenzfrequenz 0.02 Hz. 
- Hochpass, Grenzfrequenz 0.02 Hz.

Berechnen Sie das Amplitudenspektrum der gefilterten Wertereihen. Stellen Sie die Ergebnisse graphisch dar und interpretieren Sie die Ergebnisse.

### Aufgabe 7.4

Berechnen Sie einen Butterworth-Bandpass mit den Grenzfrequenzen  0.02 Hz und 0.1 Hz. Filtern Sie die Wertereihe der vorangegangenen Aufgabe von beiden Seiten. Interpretieren Sie die Ergebnisse.

Wir lösen Aufgabe 7.3 und 7.4 mit einem Pythoncode. Wie immer importieren wir als erstes die üblichen Pyhton Bibliotheken. Aus der `SciPy` Bibliothek benötigen wir einige Codes zur Definition und Anwendung von Butterworth Filtern ...

In [2]:
# Importiere Python Bibliotheken 
# ------------------------------
%matplotlib inline
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal

Ich will das Problem möglichst modular programmieren, so daß man den Code bei Bedarf auch in anderen Codes verwenden kann. In der Funktion `butt_filt` wird der Butterworth-Filter definiert und auf die Zeitreihe $x(t)$ angewendet. 

Die Parameter **ffiltmin** und **fflitmax** legen die untere und obere Grenzfrequenz für einen Bandpass-Filter fest. Im Fall eines Tiefpass oder Hochpassfilters wird ffiltmin als Grenzfrequenz verwendet. Mit dem Parameter **order** läßt sich die Ordnung des Butterworthfilters definieren.

Mit der `SciPy` Funktion [signal.butter](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.butter.html) wird der Butterworthfilter definiert und mit [signal.sosfilt](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.sosfilt.html) einseitig auf die Zeitreihe angewendet. Soll der Filter zweitseitig angewendet werden, verwendet man einfach die Funktion [signal.sosfiltfilt](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.sosfiltfilt.html). Die Schalter **low**, **high** und **band** aktivieren entweder einen Tiefpass-, Hochpass- oder Bandpass Butterworth Filter. Mit der Option **two_sided** läßt sich die zweiseitige Filterung aktivieren, per Default wird eine einseitige Filterung angewendet.  

In [3]:
def butt_filt(x,dt,ffiltmin,ffiltmax,order,low,high,band,two_sided):
    
    fs = 1./dt # sample frequency
    
    # design low-pass filter
    # ----------------------
    if(low==True):
        
        sos = signal.butter(order, ffiltmin, 'lp', fs=fs, output='sos')
            
    # design high-pass filter
    # -----------------------
    if(high==True):
        
        sos = signal.butter(order, ffiltmin, 'hp', fs=fs, output='sos')
    
    # design band-pass filter
    # -----------------------
    if(band==True):
        
        sos = signal.butter(order, [ffiltmin,ffiltmax], 'bp', fs=fs, output='sos')    
    
    # apply filter to time series x(t) either one-sided or two-sided    
    # --------------------------------------------------------------
    if(two_sided==False):
        
        x = signal.sosfilt(sos, x)     # apply one-sided filter
        
    else:
        
        x = signal.sosfiltfilt(sos, x) # apply two-sided filter        
    
    return x

Um die Zeitreihe möglichst flexibel definieren zu können, programmieren wir dafür ebenfalls eine Funktion `time_series`. Als erstes implementieren wir die Zeitreihe gemäß Aufgabe 7.3 ... 

In [4]:
def time_series():
    
    # Define parameters
    dt = .1                # time sampling [s]
    L = 200.               # length of the time series [s]
    tspike = 100.          # position of spike in time series [s]
    
    # Define time series ...
    t = np.arange(0,L+dt,dt) # compute time vector
    nt = len(t)              # number of samples in time series
    x = np.zeros(nt)         # intialize time series with zeros
    
    # Compute time sample of spike
    ntspike = (int)(tspike // dt)
    
    # Add spike to time series
    x[ntspike] = 1.
    
    return t, x

Für die Berechnung der Amplitudenspektren verwenden wir die Funktion `x_fft` zur Berechnung der FFT und Frequenzen. Diese können wir einfach aus dem Notebook zu [Übung 3](https://nbviewer.org/github/daniel-koehn/Geophysikalische-Signalverarbeitung/blob/master/exercises/Ex03_FFT.ipynb) übernehmen ...

In [5]:
def x_fft(x,dt):
    
    # Fourier transform
    X = np.fft.fft(x)
    
    # Normalize by length of the time series
    N = len(x)
    X = X / (N*dt)
    
    # estimate frequencies    
    freq = np.fft.fftfreq(N, d=dt)
    
    return freq,X

Schließlich programmieren wir die Funktion `butterworth` in der die Zeitreihe definiert, die Filter designt, auf die Zeitreihe $x(t)$ angewendet, Amplitudenspektren berechnet und  visualisiert werden ...

In [6]:
def butterworth(low,high,band,two_sided,order):
    
    # Define filter parameters
    ffiltmin = 0.02        # minimum cut-off frequency for Butterworth filter [Hz]
    ffiltmax = 0.1         # maximum cut-off frequency for Butterworth filter [Hz]
    
    # Define time series
    # ------------------
    t, x = time_series()
    nt = len(t)     # number of samples in time series
    dt = t[1]-t[0]  # sample interval [s]
    

    # Compute frequencies and spectrum of time series x(t)
    freq, X = x_fft(x,dt)
    
    # Define and apply Butterworth ...
    # --------------------------------
    if(low==True): # ... low-pass filter
        
        x_low = butt_filt(x,dt,ffiltmin,ffiltmax,order,True,False,False,two_sided)
                
        # fft of low-pass filtered time series
        freq, X_low = x_fft(x_low,dt)
        
    if(high==True): # ... high-pass filter

        x_high = butt_filt(x,dt,ffiltmin,ffiltmax,order,False,True,False,two_sided)
        
        # fft of high-pass filtered time series
        freq, X_high = x_fft(x_high,dt)
        
    if(band==True): # ... band-pass filter
        
        x_band = butt_filt(x,dt,ffiltmin,ffiltmax,order,False,False,True,two_sided)        
        
        # fft of band-pass filtered time series
        freq, X_band = x_fft(x_band,dt)    
    
    # Plot time series x(t)
    # ---------------------
    plt.figure(figsize=(20,10))
    
    plt.subplot(211)
    
    if(low==False and high==False and band==False):
        plt.plot(t, x, 'b-',lw=3,label='x(t)')
        
    if(low==True):
        plt.plot(t, x_low, 'r-',lw=3,label='low-pass x(t)')        
        
    if(high==True):
        plt.plot(t, x_high, 'g-',lw=3,label='high-pass x(t)')
        
    if(band==True):
        plt.plot(t, x_band, 'y-',lw=3,label='band-pass x(t)')    
        
    plt.title('x(t)')
    plt.xlabel('time [s]')
    plt.ylabel('(Filtered) time series x(t)')
    plt.legend()
    plt.grid()
    
    plt.subplot(212)
    
    # plot KKF
    plt.plot(freq[:nt//2], np.abs(X[:nt//2]), 'b-',lw=3,label='X(f)')
    if(low==True):
        plt.plot(freq[:nt//2], np.abs(X_low[:nt//2]), 'r-',lw=3,label='low-pass X(f)')
        
    if(high==True):
        plt.plot(freq[:nt//2], np.abs(X_high[:nt//2]), 'g-',lw=3,label='high-pass X(f)')
        
    if(band==True):
        plt.plot(freq[:nt//2], np.abs(X_band[:nt//2]), 'y-',lw=3,label='band-pass X(f)')
        
    plt.title('Amplitude spectra of (filtered) time series')
    plt.xlabel(r'frequency f [Hz]')
    plt.ylabel('Amplitude')
    plt.legend()
    plt.xlim(0,.2)
    plt.grid()

Um den Einfluß verschiedener Filter auf die Zeitreihe einfacher vergleichen zu können, verwenden wir interaktive Jupyter Widgets. Neben Schaltern für die Tief-, Hoch- und Bandpass Butterworth Filter kann auch eine zweiseitige Filterung aktiviert werden. Die Ordnung des Filters läßt sich interaktiv zwischen 1 und 10 variieren.

In [7]:
interactive_plot = interactive(butterworth, low=False, high=False, band=False, two_sided=False, order=(1,10,1))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(Checkbox(value=False, description='low'), Checkbox(value=False, description='high'), Che…

Der Code zeigt ...

- Die Anwendung der Filter führt zu der erwarteten Filterwirkung des weißen Spektrums einer Zeitreihe mit dem Dirac'schen Delta-Impuls, welche durch die Grenzfrequenzen vorgegeben werden. 

- Durch Variation der Ordnung des Filters läßt sich die Flankensteilheit des Filters einstellen. Mit größerer Flankensteilheit im Frequenzbereich treten deutlichere Side-Lobes im Zeitbereich auf. Entsprechend sollte man die Ordnung des Filters so wählen, daß ein guter Kompromiss zwischen Filterwirkung und Side-Lobes gefunden wird.

- Der einseitige Filter führt zu einer Minimum-Phasenfilterung, der zweiseitige Filter dagegen zu einer Zero-Phasefilterung des Delta-Impulses. Je nachdem wie die Daten weiterverarbeitet, analysiert und interpretiert werden sollen, kann die Wahl eines einseitigen oder zweiseitigen Filters entscheidend sein.

### Aufgabe 7.5

Verändern Sie die Wertereihe so, dass alle Werte 1 sind. Wiederholen Sie die einseitige und zweiseitige Filterung. Interpretieren Sie die Ergebnisse.

Da der Code modular programmiert wurde, müssen wir nur die Zeitreihe in der Funktion `time_series` gemäß Aufgabenstellung 7.5 ändern ... 

In [8]:
def time_series():
    
    # Define parameters
    dt = .1                # time sampling [s]
    L = 200.               # length of the time series [s]
    tspike = 100.          # position of spike in time series [s]
    
    # Define time series ...
    t = np.arange(0,L+dt,dt) # compute time vector
    nt = len(t)              # number of samples in time series
    x = np.ones(nt)         # intialize time series with ones
    
    return t, x

... und das interaktive IPython Widget neu starten ...

In [9]:
interactive_plot = interactive(butterworth, low=False, high=False, band=False, two_sided=False, order=(1,10,1))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(Checkbox(value=False, description='low'), Checkbox(value=False, description='high'), Che…

Wenn die Zeitreihe einen konstanten Wert (Bias) besitzt, ergibt sich ein Dirac'scher Deltaimpuls bei einer Frequenz von 0 Hz im Amplitudenspektrum. Entsprechend sollte ...

- ... der Tiefpassfilter die Zeitreihe nicht verändern. Das funktioniert bei dem zweiseitigen Filter sehr gut. Bei der Variation der Ordnung des Butterworthfilters treten nur teilweise etwas seltsame Achsenskalierungen auf, die versuchen kleine Abweichungen in der gefilterten Zeitreihe wiederzuspiegeln. Der einseitige Filter hat aufgrund der Minimalphasigkeit allerdings das Problem, daß die gefilterte Zeitreihe am Anfang einen Wert von Null annimmt und erst mit zunehmender Zeit den Wert 1.

- ... der Hochpass- und Bandpassfilter bei den vorgegebenen Grenzfrequenzen den Bias entfernen. Wie beim Tiefpassfilter liefert der zweiseitige Filter das gewünschte Ergebnis. Der einseitige Filter liefert hingegen stark von Null abweichende Werte zu Beginn der Zeitreihe.